## Computing rho max in terms of eta_critic - eta


In [ ]:
# Sorting the columns from minor to major, in case the density has not been stored properly

# Read the contents of the text file into a list
with open('density_max.dat', 'r') as file:
    lines = file.readlines()

# Sort the lines based on the values in the 4th column
sorted_lines = sorted(lines, key=lambda line: float(line.split()[3]))

# Write the sorted lines to a new file
with open('output_density.dat', 'w') as file:
    file.writelines(sorted_lines)


In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# Loading the data from the generated file from SFcollapse1D
phi0, density, subs, log_subs, log_den = np.loadtxt("output_density.dat").T

In [ ]:
#Generation of the runscript to compute the echoing period from the adjusted plot of the central density

import numpy as np

eta_weak = 0.3364266156435
eta_strong = 0.3364266156436
eta_c = 0.5 * (eta_weak + eta_strong)
starting_point = -22
last_point = -5.5
eta_initial = eta_c - np.exp(starting_point)
eta_final = eta_c - np.exp(last_point)
delta_x = 0.5
eta_insert = 0
length = np.int(np.abs((starting_point - last_point) * 2))

for i in range(length):
    eta_insert = eta_c - np.exp(starting_point) * np.exp(0.5 * i)
    line = f"taskset -c 0,1,2,3,4,5 ./SFcollapse1D 320 16 5.3162 0.08 {eta_insert:.13f}"
    print(line)


## FITTING THE FUNCTION 

In [ ]:
#The data obtained from the preceding sub-runs needs to be fitted with the following function.
from scipy.optimize import curve_fit
def adjust(x, C, gamma, k, w, phase):
    y = C - 2*gamma*x + k*np.sin(w*x + phase)
    return y

# A guess can be done to achieve a better result
guess = [1, 0.37, 0.1, 1.32, -1]
parameters, covariance = curve_fit(adjust, log_subs, log_den, p0 = guess)
fit_c = parameters[0]
fit_gamma = parameters[1]
fit_k = parameters[2]
fit_w = parameters[3]
fit_phase = parameters[4]

In [ ]:
#Plot of the results of the simulation together with the fitting function
import matplotlib as mpl

# Configure matplotlib to use LaTeX 
mpl.rcParams['text.usetex'] = True
mpl.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}' 


fit_function = adjust(log_subs, fit_c, fit_gamma, fit_k, fit_w, fit_phase)
plt.figure(figsize=(10, 8))  
plt.plot(log_subs, fit_function, '-', label='fit')
plt.plot(log_subs, log_den, 'o', label ='Numerical Results')
plt.legend(fontsize = 18)
plt.grid()
plt.ylabel(r'$\text{Log}(\rho^{max}_{central})$', fontsize = 16)
plt.xlabel(r'Log($\eta^* - \eta$)', fontsize = 16)
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)

#plt.show()
plt.savefig('/home/usermiguel/Desktop/changing_code_comments')

In [ ]:
# Computing the standard error
SE = np.sqrt(np.diag(covariance))
SE_C = SE[0]
SE_gamma = SE[1]
SE_k = SE[2]
SE_w = SE[3]
SE_phase = SE[4]
Delta = 4*np.pi*fit_gamma/fit_w
error_delta = 4*np.pi*(SE_gamma/fit_w + fit_w*SE_w/fit_w**2)
print(F'The value of c is {fit_c:.5f} with standard error of {SE_C:.5f}.')
print(F'The value of gamma is {fit_gamma:.5f} with standard error of {SE_gamma:.5f}.')
print(F'The value of k is {fit_k:.5f} with standard error of {SE_k:.5f}.')
print(F'The value of w is {fit_w:.5f} with standard error of {SE_w:.5f}.')
print(F'The value of phase is {fit_phase:.5f} with standard error of {SE_phase:.5f}.')
print(F'The value of Delta is {Delta:.5f} with standard error of {error_delta:.5f}.')


## Plotting the useful graphs 

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import numpy as np


# Weak data 
t_w, alp_w, sf_w = np.loadtxt("out_weak.dat").T

# Strong data
t_s, alp_s, sf_s = np.loadtxt("out_strong.dat").T

colour_w = 'red'
colour_s = 'blue'
linewidth = 1 
plt.grid(ls=':')

# Plot of the data
fig, ax = plt.subplots(figsize=(10, 6))  # Create the figure and axis

ax.plot(t_w, alp_w, lw=linewidth, c=colour_w, label=r"$\eta_{\rm weak} = 0.3364$")
ax.plot(t_s, alp_s, lw=linewidth, c=colour_s, ls='--', label=r"$\eta_{\rm strong} = 0.3365$")
ax.legend(loc=2, markerfirst=False, fontsize=14)
ax.grid()
# Set titles for axis labels
ax.set_xlabel('Time', fontsize=14)  
ax.set_ylabel(r"$\alpha_{\rm central}$", fontsize=19)  

# Increase font size of axis labels
ax.tick_params(axis='both', which='major', labelsize=12) 


# Set title for the plot
ax.set_title('Evolution of the lapse function', fontsize=16)




In [ ]:
# Plot of the graph with a more precise value of the critical constant

# Weak data 
t_w, alp_w, sf_w = np.loadtxt("out_weak.dat").T

# Strong data
t_s, alp_s, sf_s = np.loadtxt("out_strong.dat").T

colour_w = 'red'
colour_s = 'blue'
linewidth = 1 
plt.grid(ls=':')

# Plot of the data
fig, ax = plt.subplots(figsize=(10, 6))  

ax.plot(t_w, alp_w, lw=linewidth, c=colour_w, label=r"$\eta_{\rm weak} = 0.3364266156435$")
ax.plot(t_s, alp_s, lw=linewidth, c=colour_s, ls='--', label=r"$\eta_{\rm strong} =  0.3364266156436$")
ax.legend(loc=2, markerfirst=False, fontsize=14)
ax.grid()
# Set titles for axis labels
ax.set_xlabel('Time', fontsize=14)  
ax.set_ylabel(r"$\alpha_{\rm central}$", fontsize=19) 

# Increase font size of axis labels
ax.tick_params(axis='both', which='major', labelsize=12)  


# Set title for the plot
ax.set_title('Evolution of the lapse function', fontsize=16)




In [ ]:
# Plot of the graph previous graph with some zoom applied 

# Weak data 
t_w, alp_w, sf_w = np.loadtxt("out_weak.dat").T

# Strong data
t_s, alp_s, sf_s = np.loadtxt("out_strong.dat").T

colour_w = 'red'
colour_s = 'blue'
linewidth = 1 
plt.grid(ls=':')

# Plot of the code
fig, ax = plt.subplots(figsize=(10, 6))  

ax.plot(t_w, alp_w, lw=linewidth, c=colour_w, label=r"$\eta_{\rm weak} = 0.3364266156435$")
ax.plot(t_s, alp_s, lw=linewidth, c=colour_s, ls='--', label=r"$\eta_{\rm strong} = 0.3364266156436$")
ax.legend(loc=2, markerfirst=False, fontsize=14)
plt.xlim(5.304,5.318)
plt.ylim(0.0,0.15)
ax.grid()
# Set titles for axis labels
ax.set_xlabel('Time', fontsize=14)  
ax.set_ylabel(r"$\alpha_{\rm central}$", fontsize=19)  
# Increase font size of axis labels
ax.tick_params(axis='both', which='major', labelsize=12) 


# Set title for the plot
ax.set_title('Zoom in to analyse the late evolution', fontsize=16)




## Plot to obtain the scalar field itself

In [ ]:
import matplotlib.pyplot as plt

def plot_sf_i_vs_t_i(t, sf, color, label):
    plt.plot(t, sf, marker='o', linestyle='-', color=color, label=label, linewidth=0.2)  # Adjust linewidth here

# Load data
t_w, alp_w, sf_w = np.loadtxt("out_weak.dat").T
t_s, alp_s, sf_s = np.loadtxt("out_strong.dat").T

# Set plot parameters
colour_w = 'red'
colour_s = 'blue'
linewidth = 1
# Plot weak data
plt.plot(t_w, sf_w, lw=linewidth, c=colour_w, label=r"$\eta_{\rm weak} = 0.3364266156435$")

# Plot strong data
plt.plot(t_s, sf_s, lw=linewidth, c=colour_s, label=r"$\eta_{\rm strong} = 0.3364266156436$")

plt.xlabel('t_i')  # Label for the x-axis
plt.ylabel('sf_i')  # Label for the y-axis
plt.title('Plot of sf_i vs t_i')  # Title for the plot
plt.grid(ls=':')  # Enable grid with dotted linestyle
plt.legend()  # Show legend
plt.show()  # Display the plot
